In [1]:
%pip install python-dotenv
from dotenv import load_dotenv
import os 
load_dotenv()

Note: you may need to restart the kernel to use updated packages.


True

In [2]:
%pip install langchain
%pip install langchain_together

Note: you may need to restart the kernel to use updated packages.
Note: you may need to restart the kernel to use updated packages.


In [4]:

from langchain_together import ChatTogether, TogetherEmbeddings
from langchain.prompts import ChatPromptTemplate, MessagesPlaceholder
from langchain.schema import SystemMessage
from langchain.memory import ConversationBufferMemory
from langchain.chains import LLMChain

In [ ]:

llm = ChatTogether(
    model="deepseek-ai/DeepSeek-V3",
    temperature=0,
    streaming=True,
)

In [ ]:
system_message = SystemMessage(
    content="""You are Autobot, an IITM BS degree virtual assistant designed to help students navigate the BS Data Science program.  

    Your role is to:  
    - Assist students with understanding course concepts and guidelines.  
    - Provide explanations, hints, and study strategies without giving direct answers.  
    - Help students locate relevant sections in their course materials.  
    - Encourage academic integrity by guiding students toward self-learning.  

    **Guidelines:**  
    - DO NOT provide direct answers to assignments, quizzes, or exams.  
    - DO NOT generate or suggest additional resources—only the TA or instructor bot can do that.  
    - Use only the retrieved context to answer questions. If the information is unavailable, say, "I don't have that information."  
    - If a student asks for a resource, politely inform them that TAs or instructors can provide those.  
    - Maintain a professional yet supportive tone to encourage learning.  

    Your goal is to **assist students in learning while maintaining academic integrity**.  
    """
)


In [ ]:
prompt = ChatPromptTemplate.from_messages([
    system_message,  # System prompt
    MessagesPlaceholder(variable_name="retrieved_context"),
    ("human", "{user_query}") 
])

In [ ]:
prompt

In [ ]:
autobot = prompt | llm

In [ ]:
%pip install -U langchain-community

In [ ]:

from langchain.vectorstores import FAISS
from langchain.embeddings import HuggingFaceEmbeddings
from langchain.document_loaders import TextLoader
from langchain.text_splitter import RecursiveCharacterTextSplitter

In [ ]:
loader = TextLoader("./assets/Intro-to-python.md")
documents = loader.load()



In [ ]:
text_splitter = RecursiveCharacterTextSplitter(
    chunk_size = 720 , chunk_overlap = 72
)
chunks = text_splitter.split_documents(documents)

In [ ]:
chunks[0].page_content

In [ ]:
embedding_model = TogetherEmbeddings()

In [ ]:
vector_store = FAISS.from_documents(chunks , embedding_model)

In [ ]:

def reterive_context(user_query):
    
    docs = vector_store.similarity_search(user_query, k=3)
    content =  "\n".join([doc.page_content for doc in docs])
    return [SystemMessage(content=content)]


In [ ]:
print(reterive_context("Can you tell me about the grading of the python course"))

In [ ]:
def call_autobot(user_query,reterived_context):
   return autobot.invoke({"user_query":user_query,"retrieved_context":reterived_context})
    

In [ ]:
user_query="i want add a resource on variable"
reterived_context = reterive_context(user_query)
response = call_autobot(user_query,reterived_context)
print(response.content)